In [ ]:
import os
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt
import tensorflow as tf
import wandb
from wandb.integration.keras import WandbMetricsLogger, WandbModelCheckpoint

%run helpFunction.ipynb

In [ ]:
import sys
sys.version

In [ ]:
gpus = tf.config.list_physical_devices('GPU')
if gpus:
  try:
    # Currently, memory growth needs to be the same across GPUs
    for gpu in gpus:
      tf.config.experimental.set_memory_growth(gpu, True)
    logical_gpus = tf.config.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Memory growth must be set before GPUs have been initialized
    print(e)

In [ ]:
run = wandb.init(project="VisionTransformer", entity="@ass")

In [ ]:
# Load Dataset
artifact = run.use_artifact(
    "@ass/VisionTransformer/ecg-image-data-no-preprocessing:v0",
    type="dataset",
)

artifact_dir = artifact.download()

# Get Training Data
training_artifact = np.load(
    os.path.join(artifact_dir, "training-no-preprocessing.npy")
)
training_images = training_artifact["images"]
training_labels = training_artifact["labels"]

# Get Validation Data
validation_artifact = np.load(
    os.path.join(artifact_dir, "validation-no-preprocessing.npy")
)
validation_images = validation_artifact["images"]
validation_labels = validation_artifact["labels"]

# Get Test Data
test_artifact = np.load(os.path.join(artifact_dir, "test-no-preprocessing.npy"))
test_images = test_artifact["images"]
test_labels = test_artifact["labels"]

In [ ]:
plt.imshow(training_images[0]), training_labels[0]

In [ ]:
training_images.shape, test_images.shape, validation_images.shape

In [ ]:
training_labels.shape

In [ ]:
train_datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

test_datagen = ImageDataGenerator()

In [ ]:
batch_size = 32

In [ ]:
train_generator = train_datagen.flow(training_images, training_labels, batch_size=batch_size)
test_generator = test_datagen.flow(test_images, test_labels, batch_size=batch_size)
validation_generator = test_datagen.flow(validation_images, validation_labels, batch_size=batch_size)

In [ ]:
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(250, 250, 3)),
    MaxPooling2D(2, 2),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(1, activation='sigmoid')
])


In [ ]:
model.compile(optimizer=Adam(), loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
history = model.fit(
    train_generator,
    steps_per_epoch=len(training_images) // batch_size,
    epochs=100,
    validation_data=validation_generator,
    validation_steps=len(validation_images) // batch_size,
    callbacks=[WandbMetricsLogger()]
)

In [ ]:
# Modell evaluieren
test_loss, test_acc = model.evaluate(test_images, test_labels)
print(f"Testgenauigkeit: {test_acc:.4f}, Testverlust: {test_loss:.4f}")

In [ ]:
wandb.log({'test_loss': test_loss, "test_accc": test_acc})

In [ ]:
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')

plt.subplot(1, 2, 2)
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')

plt.suptitle('Model Training - Basic CNN')
plt.tight_layout()
plt.show()


In [ ]:
predictions = model.predict(test_images)
predictions = (predictions > 0.5)

prediction_csv(test_labels, predictions, 'Basic CNN')
print(classification_report(test_labels, predictions))

In [ ]:
namelabels = {0: 'Wartelinie', 1: 'keine Wartelinie'}
cm = confusion_matrix(test_labels, predictions)
sns.heatmap(cm, annot=True, cmap='Blues', fmt='d', xticklabels=namelabels.values(), yticklabels=namelabels.values())
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.title('Confusion Matrix - Basic CNN Model')
wandb.log({"confusion-matrix": wandb.Image(plt)})
plt.show()

In [ ]:
wandb.finish()